In [29]:
import os
from datetime import datetime, timedelta
import pandas as pd
from zeep import Client
import loadenv
import db
from db import get_connection
import psycopg2.extras
import logging
import sys

logger = logging.getLogger(__name__)
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

In [32]:
def parse_and_insert_sheet(df, sheet_name, col_name):
    logger.info(f"Parse sheet {sheet_name}")

    stand = df[sheet_name].iloc[1, 0]

    datenbestand = datetime.strptime(stand, 'Stand: %d.%m.%Y %H:%M:%S')

    nan_value = float("NaN")
    data = df[sheet_name].iloc[4:,2:].replace("", nan_value).dropna(how='all', axis=1)
    
    col = 0
    for d in data.loc[data.index[0]]:
        if isinstance(d, str) and d != '':
            try:
                parsed = datetime.strptime(d, '%d.%m.%Y')

                data.iat[0, col] = parsed
            except ValueError as _:
                pass

        col += 1

    # data[2] = data[2].apply(lambda x: "{:05d}".format(x) if isinstance(x, int) else x)

    entries = []
    for r_idx in range(1, data.shape[0]):
        for c_idx in range(3, data.shape[1]):
            val = None if data.iloc[r_idx, c_idx] == '' else data.iloc[r_idx, c_idx]
            entry = {
                'datenbestand': datenbestand,
                'id': "{:05d}".format(data.iloc[r_idx, 2]),
                'ts': data.iloc[0, c_idx],
                'val': val
            }
            entries.append(entry)
            
    return entries

#     psycopg2.extras.execute_values(
#         cur,
#         f"""INSERT INTO 
#             rki_incidence_excel (datenbestand, ags, timestamp, \"{col_name}\") 
#             VALUES %s ON CONFLICT ON CONSTRAINT rki_incidence_excel_pk DO
#             UPDATE SET
#                 \"{col_name}\" = EXCLUDED.\"{col_name}\",
#                 updated_at = NOW();
#         """,
#         entries,
#         template='(%(datenbestand)s, %(id)s, %(ts)s, %(val)s)',
#         page_size=500
#     )
#     conn.commit()

    return True


def parse_and_insert(fp: str) -> bool:
    df = pd.read_excel(fp, sheet_name=['LK_7-Tage-Inzidenz', 'LK_7-Tage-Fallzahlen'], header=None,
                       na_filter=False, engine="openpyxl")

    parse_and_insert_sheet(df=df, sheet_name='LK_7-Tage-Inzidenz', col_name='7_day_incidence')
    parse_and_insert_sheet(df=df, sheet_name='LK_7-Tage-Fallzahlen', col_name='7_day_cases')

    return True

fp = './Fallzahlen_Kum_Tab.xlsm'
df = pd.read_excel(fp, sheet_name=['LK_7-Tage-Inzidenz', 'LK_7-Tage-Fallzahlen'], header=None,
                       na_filter=False, engine="openpyxl")

nan_value = float("NaN")

data = df['LK_7-Tage-Fallzahlen'].iloc[4:,2:].replace("", nan_value).dropna(how='all', axis=1)

parse_and_insert_sheet(df, 'LK_7-Tage-Fallzahlen', 'egal')

INFO:__main__:Parse sheet LK_7-Tage-Fallzahlen


[{'datenbestand': datetime.datetime(2021, 5, 2, 7, 43, 18),
  'id': '00999',
  'ts': datetime.datetime(2020, 11, 20, 0, 0),
  'val': 971},
 {'datenbestand': datetime.datetime(2021, 5, 2, 7, 43, 18),
  'id': '00999',
  'ts': datetime.datetime(2020, 11, 21, 0, 0),
  'val': 899},
 {'datenbestand': datetime.datetime(2021, 5, 2, 7, 43, 18),
  'id': '00999',
  'ts': datetime.datetime(2020, 11, 22, 0, 0),
  'val': 675},
 {'datenbestand': datetime.datetime(2021, 5, 2, 7, 43, 18),
  'id': '00999',
  'ts': datetime.datetime(2020, 11, 23, 0, 0),
  'val': 646},
 {'datenbestand': datetime.datetime(2021, 5, 2, 7, 43, 18),
  'id': '00999',
  'ts': datetime.datetime(2020, 11, 24, 0, 0),
  'val': 832},
 {'datenbestand': datetime.datetime(2021, 5, 2, 7, 43, 18),
  'id': '00999',
  'ts': datetime.datetime(2020, 11, 25, 0, 0),
  'val': 758},
 {'datenbestand': datetime.datetime(2021, 5, 2, 7, 43, 18),
  'id': '00999',
  'ts': datetime.datetime(2020, 11, 26, 0, 0),
  'val': 648},
 {'datenbestand': datetime.